# Modelos de Contagem, Tobit e Seleção Amostral

Prof. Daniel de Abreu Pereira Uhr

### Conteúdo

* Modelos de Contagem
  * Poisson Model
  * Negative Binomial Model
  * Zero-Inflated Poisson (ZIP)
  * Zero Inflated Negative Binomial Model (ZINB)
* Modelo Tobit
* Modelo de Seleção Amostral


**Referências**

* Cameron e Triverdi – Microeconometrics using stata. 
  * Capítulo 16 – Tobit and Selection models
  * Capítulo 17 – Count Data Models
* Cameron e Triverdi – Microeconometrics, Methods and Aplication
  * Capítulo 16 – Tobit and Selection models




---

## Modelos de Contagem

### Modelo de Poisson

Em muitos contextos de pesquisa, a variável de resultado $Y$ é uma contagem, ou seja, um número inteiro não negativo. Exemplos comuns incluem o número de chamadas recebidas por um call center, o número de acidentes de trânsito em uma determinada interseção, ou o número de filhos em uma família. Nesses casos, a variável $Y$ pertence ao conjunto $\mathbb{N}_0 = \{0, 1, 2, 3, \ldots \}$. O objetivo é modelar $Y$ como uma função das variáveis explicativas $X$ através de uma estrutura de regressão.

Uma modelagem natural para esses dados de contagem é o **Modelo de Poisson**, que assume que o número de ocorrências de um evento segue uma distribuição de Poisson.

#### Função de Probabilidade da Distribuição de Poisson

A distribuição de Poisson é definida para modelar a probabilidade do número de ocorrências de um evento em um intervalo fixo de tempo ou espaço, dado que esses eventos ocorrem com uma taxa média constante e independentemente do tempo desde o último evento. A função de probabilidade para $Y$, o número de ocorrências, é dada por:

$$ \Pr(Y = y) = \frac{\lambda^y e^{-\lambda}}{y!}, \quad y = 0, 1, 2, \ldots $$

Onde:
- $\lambda$ é o parâmetro da distribuição, que representa tanto a média quanto a variância do número de ocorrências.
- $e$ é a base do logaritmo natural (aproximadamente 2,718).
- $y!$ é o fatorial de $y$, ou seja, o produto de todos os inteiros positivos até $y$.

#### Momentos da Distribuição de Poisson

Os momentos da distribuição de Poisson são dados por:

- **Esperança (Média):**
  $$ E(Y) = \lambda $$

- **Variância:**
  $$ \text{Var}(Y) = \lambda $$

Um aspecto notável da distribuição de Poisson é que a média e a variância são iguais. Isso contrasta com muitos modelos de regressão tradicionais, onde a variância é assumida constante.

#### Parametrização do Modelo de Poisson

No contexto de regressão, queremos modelar o parâmetro $\lambda$ como uma função das variáveis explicativas $X$. Para garantir que $\lambda$ seja sempre positivo (já que é uma taxa de ocorrência), utilizamos a função exponencial:

$$ \lambda = \exp(x' \beta) $$

Onde:
- $x'$ é o vetor transposto das variáveis explicativas.
- $\beta$ é o vetor de coeficientes a serem estimados.

Assim, a esperança condicional de $Y$ dado $X$ é:

$$ E(Y|X) = \lambda = \exp(x' \beta) $$

E, como a variância também é igual a $\lambda$:

$$ \text{Var}(Y|X) = \lambda = \exp(x' \beta) $$

#### Interpretação dos Coeficientes no Modelo de Poisson

Os coeficientes $\beta_j$ no modelo de Poisson têm uma interpretação multiplicativa. Mais especificamente, o exponencial de $\beta_j$ representa a mudança proporcional na média esperada de $Y$ para um aumento unitário em $x_j$, mantendo as outras variáveis constantes.

Por exemplo, se $\beta_j = 0.2$, então $\exp(0.2) \approx 1.22$, o que indica que um aumento unitário em $x_j$ está associado a um aumento de 22% na média esperada de $Y$.

#### Heterocedasticidade no Modelo de Poisson

O modelo de Poisson é **intrinsecamente heterocedástico**, o que significa que a variância de $Y$ não é constante, mas varia com o valor esperado $\lambda$. À medida que $\lambda$ aumenta (por exemplo, conforme aumentam as variáveis $X$), a variância de $Y$ também aumenta. Isso contrasta com os modelos de regressão linear clássicos, que assumem homocedasticidade (variância constante dos erros).

#### Exemplo Prático de Aplicação do Modelo de Poisson

Suponha que estamos interessados em modelar o número de chamadas recebidas por um call center em um dia típico. As variáveis explicativas podem incluir o número de agentes disponíveis, o dia da semana e a época do ano. Usando o modelo de Poisson, podemos estimar a relação entre essas variáveis e o número esperado de chamadas.

Se o coeficiente estimado para o número de agentes disponíveis for positivo, isso indicaria que, conforme o número de agentes aumenta, esperamos um maior número de chamadas recebidas, refletido no aumento do parâmetro $\lambda$.

#### Limitações do Modelo de Poisson

Embora o modelo de Poisson seja útil em muitas situações, ele tem algumas limitações:

1. **Superdispersão**: O modelo de Poisson assume que a média e a variância são iguais. No entanto, em muitos conjuntos de dados reais, a variância é maior do que a média, um fenômeno conhecido como superdispersão. Quando há superdispersão, o modelo de Poisson pode subestimar a variabilidade dos dados, levando a erros-padrão subestimados e testes de hipóteses inválidos.

2. **Subdispersão**: Menos comum que a superdispersão, mas ocorre quando a variância é menor que a média, o que também não é capturado adequadamente pelo modelo de Poisson.

#### Alternativas ao Modelo de Poisson

Quando a suposição de equidispersão (igualdade entre média e variância) não é adequada, outras abordagens podem ser utilizadas, como:

- **Modelo Binomial Negativo**: Extensão do modelo de Poisson que permite que a variância seja maior que a média.
- **Modelos de Poisson Hurdle e Zero-Inflated Poisson (ZIP)**: Usados para lidar com um excesso de zeros nas contagens, comum em muitos conjuntos de dados.


In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats import chi2

In [2]:
# DataFrame
df = pd.read_stata("https://github.com/Daniel-Uhr/data/raw/main/cattaneo2.dta")

In [3]:
# Ajustando as variáveis
# Criar a variável de resultado de contagem
df['Y'] = df['nprenatal']

In [4]:
# Definindo a fórmula da regressão de Poisson
formula = 'Y ~ medu + fedu + mage + medu + deadkids + alcohol'

# Ajustando o modelo de Poisson
poisson_model = smf.poisson(formula=formula, data=df).fit()

# Exibindo o resumo do modelo
print(poisson_model.summary())


Optimization terminated successfully.
         Current function value: 2.765307
         Iterations 5
                          Poisson Regression Results                          
Dep. Variable:                      Y   No. Observations:                 4642
Model:                        Poisson   Df Residuals:                     4636
Method:                           MLE   Df Model:                            5
Date:                Thu, 08 Aug 2024   Pseudo R-squ.:                 0.01619
Time:                        16:05:31   Log-Likelihood:                -12837.
converged:                       True   LL-Null:                       -13048.
Covariance Type:            nonrobust   LLR p-value:                 4.108e-89
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.8943      0.027     69.755      0.000       1.841       1.947
medu           0.0151      0.

Para calcular os efeitos marginais, temos duas opções:
* **Overall**: calcula os efeitos marginais para cada observação individualmente e, em seguida, tira a média desses efeitos marginais. Isso dá uma média dos efeitos marginais observados na amostra.
* **Mean**: fixa todas as covariáveis em seus valores médios. Isto é, o efeito marginal é calculado assumindo que todas as variáveis independentes estão em seus valores médios.

In [5]:
# Calculando os efeitos marginais
marginal_effects1 = poisson_model.get_margeff()

# Exibindo o resumo dos efeitos marginais
print(marginal_effects1.summary())

       Poisson Marginal Effects      
Dep. Variable:                      Y
Method:                          dydx
At:                           overall
                dy/dx    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
medu           0.1629      0.025      6.549      0.000       0.114       0.212
fedu           0.1371      0.017      8.111      0.000       0.104       0.170
mage           0.0556      0.010      5.690      0.000       0.036       0.075
deadkids      -0.2163      0.112     -1.930      0.054      -0.436       0.003
alcohol       -1.2157      0.290     -4.189      0.000      -1.784      -0.647


Os efeitos marginais fornecem uma interpretação adicional dos coeficientes do modelo de Poisson, mostrando como a variável dependente média muda com uma mudança unitária nas variáveis independentes.

In [6]:
marginal_effects2 = poisson_model.get_margeff(at='mean')
print(marginal_effects2.summary())

       Poisson Marginal Effects      
Dep. Variable:                      Y
Method:                          dydx
At:                              mean
                dy/dx    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
medu           0.1622      0.025      6.555      0.000       0.114       0.211
fedu           0.1365      0.017      8.123      0.000       0.104       0.169
mage           0.0554      0.010      5.694      0.000       0.036       0.074
deadkids      -0.2153      0.112     -1.930      0.054      -0.434       0.003
alcohol       -1.2104      0.289     -4.191      0.000      -1.776      -0.644


### Sobredispersão no Modelo de Poisson

Uma limitação importante do modelo de Poisson é a suposição de que a média e a variância da variável dependente são iguais, o que é conhecido como **equidispersão**. No entanto, em muitos conjuntos de dados reais, a variância dos dados é maior do que a média, um fenômeno conhecido como **sobredispersão**. Quando há sobredispersão, o modelo de Poisson padrão pode produzir estimativas inadequadas, subestimando a variabilidade dos dados, o que leva a erros-padrão subestimados e, consequentemente, a testes de hipóteses inválidos.

#### Teste de Sobredispersão

Após estimar uma regressão de Poisson, é essencial testar se há sobredispersão nos dados. Um teste comum para isso envolve verificar se a variância condicional $ \text{Var}(y|x) $ é maior do que a média condicional $ \mu_i $. A forma especificada da sobredispersão pode ser expressa da seguinte maneira:

$$ \text{Var}(y|x) = \mu_i + \alpha g(\mu_i) $$

Onde:
- $ \alpha $ é um parâmetro desconhecido que captura a magnitude da sobredispersão.
- $ g(\cdot) $ é uma função conhecida, que muitas vezes assume a forma $ g(\mu_i) = \mu_i^2$.

#### Hipóteses do Teste:

- **Hipótese nula (H0):** $ \alpha = 0 $, o que implica que $ \text{Var}(y|x) = \mu_i $. Ou seja, não há sobredispersão, e o modelo de Poisson é apropriado.
- **Hipótese alternativa (H1):** $ \alpha \ne 0 $, indicando que $ \text{Var}(y|x) $ excede $ \mu_i $, sugerindo a presença de sobredispersão.

Se o teste indicar a presença de sobredispersão (ou seja, $ \alpha $ é significativamente diferente de zero), o modelo de Poisson padrão não será adequado. Neste caso, pode-se recorrer a:

1. **Modelo de Poisson robusto à heterocedasticidade**: Ajusta os erros-padrão para lidar com a variabilidade excessiva.
2. **Modelo Binomial Negativo**: Um modelo alternativo que relaxa a suposição de equidispersão, permitindo que a variância seja maior que a média.

#### Implementação do Teste de Sobredispersão

Abaixo está um exemplo em Python para calcular a estatística de sobredispersão usando os resíduos de Pearson após ajustar um modelo de Poisson:


In [7]:
# Calculando os resíduos Pearson
pearson_residuals = poisson_model.resid_pearson

# Calculando a estatística de sobredispersão
dispersion_statistic = np.sum(pearson_residuals**2) / poisson_model.df_resid

# Calculando o p-valor
p_value = 1 - chi2.cdf(dispersion_statistic, df=poisson_model.df_resid)

print(f"Estatística de sobredispersão: {dispersion_statistic}")
print(f"p-valor: {p_value}")

Estatística de sobredispersão: 1.2272192020835233
p-valor: 1.0


### Modelo Binomial Negativo

Quando trabalhamos com variáveis dependentes de contagem não negativa, é comum utilizar o modelo de Poisson para modelar a relação entre a variável resposta e as variáveis explicativas. No entanto, uma limitação significativa do modelo de Poisson é a suposição de equidispersão, onde a variância da contagem é igual à média. Em muitos casos reais, a variância excede a média, uma situação conhecida como **sobredispersão**. Para lidar com a sobredispersão, utilizamos o **Modelo Binomial Negativo**.

#### Momentos da Distribuição Binomial Negativa

Os primeiros dois momentos da distribuição binomial negativa são:

$$ E(y|x) = \mu $$

$$ \text{Var}(y|x) = \mu(1 + \alpha \mu) $$

Aqui, a variância excede a média, indicando que $\alpha > 0$ e $\mu > 0$. A sobredispersão surge porque, ao contrário do modelo de Poisson, há uma heterogeneidade não observada que afeta a contagem de eventos. Essa heterogeneidade pode ser modelada como uma variação multiplicativa no parâmetro $\lambda$ do modelo de Poisson.

#### Heterogeneidade Não Observada

A sobredispersão pode ser modelada assumindo que a contagem $y$ segue uma distribuição de Poisson condicional a uma variável latente de heterogeneidade $v$. Se a variável latente $v$ segue uma distribuição gamma, a contagem $y$ tem uma distribuição binomial negativa marginal:

$$ y_j \sim \text{Poisson}(\mu_j) $$

$$ \mu_j = \exp(x_j' \beta + \text{offset}_j + v_j) = \exp(x_j' \beta + \text{offset}_j) \cdot \exp(v_j) $$

Onde:
- **Offset**: Um parâmetro de ajuste associado à exposição (como tempo ou área).
- $e^{v_j} \sim \text{Gamma}(1/\alpha, \alpha)$: Aqui, a distribuição gamma modela a variabilidade não observada, com $\alpha$ sendo o parâmetro de sobredispersão.

Na parametrização gamma, a distribuição $\text{Gamma}(a, b)$ tem esperança $a \cdot b$ e variância $a \cdot b^2$. Quando $\alpha = 0$, o modelo binomial negativo se reduz ao modelo de Poisson.

#### Parametrizações NB1 e NB2

No modelo binomial negativo, podemos ajustar diferentes parametrizações da variância:

- **NB1**: Assume que a variância é constante para todas as observações, dada por $\text{Var}(y_j) = \mu(1 + \delta)$. Este modelo é útil em contextos onde a dispersão é constante.
  
- **NB2**: Assume que a variância é uma função quadrática da média, dada por $\text{Var}(y_j) = \mu(1 + \alpha \mu)$. Este modelo é mais flexível e captura melhor a variação da dispersão com a média, sendo a parametrização mais comum.

#### Modelos Generalizados de Binomial Negativo

No modelo binomial negativo generalizado, o parâmetro $\alpha$ pode ser modelado como uma função das covariáveis, permitindo que a sobredispersão varie entre diferentes grupos ou condições. Especificamente, podemos modelar $\ln(\alpha)$ como uma função linear das covariáveis, proporcionando maior flexibilidade no ajuste dos dados.



In [54]:
# Ajustando o modelo Binomial Negativa com dispersão constante (NB1)
nb1_model = smf.negativebinomial(formula=formula, data=df, loglike_method='nb1').fit()

print("\nModelo Binomial Negativa com Dispersão Constante (NB1):")
print(nb1_model.summary())

Optimization terminated successfully.
         Current function value: 2.753363
         Iterations: 18
         Function evaluations: 26
         Gradient evaluations: 26

Modelo Binomial Negativa com Dispersão Constante (NB1):
                     NegativeBinomial Regression Results                      
Dep. Variable:                      Y   No. Observations:                 4642
Model:               NegativeBinomial   Df Residuals:                     4636
Method:                           MLE   Df Model:                            5
Date:                qua, 07 ago 2024   Pseudo R-squ.:                 0.01462
Time:                        14:58:37   Log-Likelihood:                -12781.
converged:                       True   LL-Null:                       -12971.
Covariance Type:            nonrobust   LLR p-value:                 8.342e-80
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------

In [55]:
# Ajustando o modelo Binomial Negativa com dispersão média (NB2)
nb2_model = smf.negativebinomial(formula=formula, data=df, loglike_method='nb2').fit()

print("\nModelo Binomial Negativa com Dispersão Média (NB2):")
print(nb2_model.summary())

Optimization terminated successfully.
         Current function value: 2.757636
         Iterations: 21
         Function evaluations: 24
         Gradient evaluations: 24

Modelo Binomial Negativa com Dispersão Média (NB2):
                     NegativeBinomial Regression Results                      
Dep. Variable:                      Y   No. Observations:                 4642
Model:               NegativeBinomial   Df Residuals:                     4636
Method:                           MLE   Df Model:                            5
Date:                qua, 07 ago 2024   Pseudo R-squ.:                 0.01309
Time:                        14:58:40   Log-Likelihood:                -12801.
converged:                       True   LL-Null:                       -12971.
Covariance Type:            nonrobust   LLR p-value:                 2.922e-71
                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------

In [56]:
# Comparando os modelos usando o AIC
print(f"\nPoisson AIC: {poisson_model.aic}")
print(f"NB1 AIC (Dispersão Constante): {nb1_model.aic}")
print(f"NB2 AIC (Dispersão Média): {nb2_model.aic}")


Poisson AIC: 25685.106583294248
NB1 AIC (Dispersão Constante): 25576.217835521515
NB2 AIC (Dispersão Média): 25615.895551374393


Um AIC mais baixo indica um melhor ajuste do modelo. além disso, A Binomial Negativa é um modelo que ajusta essa sobredispersão introduzindo o parâmetro $\alpha$. Um valor positivo de $\alpha$ indica que há sobredispersão nos dados. Quanto maior o valor de $\alpha$, maior a discrepância entre a variância e a média da variável dependente, indicando maior sobredispersão.
Este valor positivo de $\alpha$ sugere que há sobredispersão nos dados. A variância da variável dependente é maior do que a média, e a Binomial Negativa está ajustando isso melhor do que um modelo de Poisson simples.

### Zero-Inflated Poisson (ZIP)

O **Modelo Zero-Inflated Poisson (ZIP)** é uma extensão do modelo de Poisson que é particularmente útil quando os dados de contagem apresentam um número excessivo de zeros, ou seja, quando há mais zeros nos dados do que seria esperado sob uma distribuição de Poisson padrão. O modelo ZIP combina dois processos: um processo que gera apenas zeros e um processo que gera contagens de acordo com uma distribuição de Poisson.

### Estrutura do Modelo ZIP

O modelo ZIP considera que a variável dependente $Y$ pode ser gerada por dois estados distintos:

1. **Estado Zero-Inflacionado**: Um estado em que a única possível observação é zero. Esse estado é modelado por uma distribuição Bernoulli.
2. **Estado Poisson**: Um estado em que a contagem é gerada de acordo com uma distribuição de Poisson tradicional.

### Componentes do Modelo ZIP

- **Probabilidade de Zero-Inflacionamento** ($\pi$): Representa a probabilidade de que a observação esteja no estado zero-inflacionado.
- **Distribuição Poisson** ($\lambda$): Representa a taxa média de ocorrência de eventos na distribuição Poisson, caso a observação não esteja no estado zero-inflacionado.

A função de probabilidade do modelo ZIP é dada por:

$$
\Pr(Y = y) = 
\begin{cases} 
\pi + (1 - \pi) \cdot \exp(-\lambda), & \text{se } y = 0 \\
(1 - \pi) \cdot \frac{\lambda^y \exp(-\lambda)}{y!}, & \text{se } y > 0 
\end{cases}
$$

Onde:
- $\pi$ é a probabilidade de observar um zero inflacionado (estado 1).
- $\lambda$ é o parâmetro da distribuição Poisson no estado 2.

### Interpretação dos Componentes

- **$\pi$**: Modela a probabilidade de ocorrência de um zero excessivo, que não é capturado pela distribuição de Poisson.
- **$\lambda$**: Modela a taxa de contagem para a distribuição Poisson quando a observação não está no estado zero-inflacionado.

### Estimação do Modelo ZIP

A estimação do modelo ZIP envolve a maximização da função de verossimilhança para ambos os componentes simultaneamente. Os parâmetros $\pi$ e $\lambda$ podem ser modelados como funções das covariáveis $X$:

- **$\pi$**: A probabilidade de inflacionamento de zeros pode ser modelada como uma função logística das covariáveis: 
  $$
  \pi = \frac{\exp(z' \gamma)}{1 + \exp(z' \gamma)}
  $$
  Onde $z$ são as covariáveis que afetam a probabilidade de inflacionamento e $\gamma$ são os coeficientes a serem estimados.

- **$\lambda$**: A média da distribuição de Poisson pode ser modelada como uma função exponencial das covariáveis:
  $$
  \lambda = \exp(x' \beta)
  $$
  Onde $x$ são as covariáveis que afetam a contagem e $\beta$ são os coeficientes a serem estimados.

### Aplicações Típicas do Modelo ZIP

O modelo ZIP é utilizado em situações onde há muitos zeros e as contagens positivas podem ser modeladas por uma distribuição de Poisson. Exemplos incluem:

- **Número de visitas ao médico**: Alguns indivíduos podem nunca visitar o médico, enquanto outros o fazem regularmente, o que cria um excesso de zeros.
- **Número de acidentes em um cruzamento**: Muitos cruzamentos podem não ter acidentes em um determinado período, mas aqueles que têm acidentes podem seguir uma distribuição de Poisson.
- **Número de compras online**: Alguns usuários podem nunca realizar compras online, resultando em zeros inflacionados.

### Limitações do Modelo ZIP

Apesar de sua utilidade, o modelo ZIP tem algumas limitações:

1. **Assunção de Independência**: O modelo assume que os zeros no estado zero-inflacionado são gerados de maneira independente das contagens no estado Poisson, o que pode não ser realista em todos os contextos.
2. **Complexidade na Interpretação**: Interpretar os resultados de um modelo ZIP pode ser mais complexo devido à mistura de dois processos subjacentes.

### Alternativas ao Modelo ZIP

Se o modelo ZIP não for adequado, outras alternativas podem ser consideradas:

- **Modelo Zero-Inflated Binomial Negativo (ZINB)**: Uma extensão do modelo ZIP que é útil quando há sobredispersão adicional nos dados além do excesso de zeros.
- **Modelo Hurdle**: Um modelo que separa o processo de geração de zeros do processo que gera contagens positivas, mas de forma diferente do ZIP.



### Zero Inflated Negative Binomial Model (ZINB)

O **Modelo Zero-Inflated Negative Binomial (ZINB)** é uma extensão do modelo Zero-Inflated Poisson (ZIP) que combina a modelagem de contagens com a capacidade de lidar tanto com o excesso de zeros quanto com a sobredispersão. Enquanto o modelo ZIP assume que as contagens positivas seguem uma distribuição de Poisson, o modelo ZINB assume que essas contagens seguem uma distribuição binomial negativa, o que permite uma maior flexibilidade ao modelar dados em que a variância é maior que a média.

### Estrutura do Modelo ZINB

Assim como o modelo ZIP, o modelo ZINB considera que a variável dependente $Y$ pode ser gerada por dois processos distintos:

1. **Estado Zero-Inflacionado**: Um estado em que a única possível observação é zero. Esse estado é modelado por uma distribuição Bernoulli.
2. **Estado Binomial Negativo**: Um estado em que a contagem é gerada de acordo com uma distribuição binomial negativa, permitindo que a variância das contagens seja maior que a média (sobredispersão).

### Componentes do Modelo ZINB

- **Probabilidade de Zero-Inflacionamento** ($\pi$): Representa a probabilidade de que a observação esteja no estado zero-inflacionado.
- **Distribuição Binomial Negativa** ($\lambda$): Representa a taxa média de ocorrência de eventos na distribuição binomial negativa, caso a observação não esteja no estado zero-inflacionado.

A função de probabilidade do modelo ZINB é dada por:

$$
\Pr(Y = y) = 
\begin{cases} 
\pi + (1 - \pi) \cdot \text{NB}(0|\lambda, \alpha), & \text{se } y = 0 \\
(1 - \pi) \cdot \text{NB}(y|\lambda, \alpha), & \text{se } y > 0 
\end{cases}
$$

Onde:
- $\pi$ é a probabilidade de observar um zero inflacionado (estado 1).
- $\lambda$ é o parâmetro da distribuição binomial negativa no estado 2.
- $\alpha$ é o parâmetro de dispersão da distribuição binomial negativa, que controla o grau de sobredispersão.

### Interpretação dos Componentes

- **$\pi$**: Modela a probabilidade de ocorrência de um zero inflacionado, que não é capturado pela distribuição binomial negativa.
- **$\lambda$**: Modela a taxa de contagem para a distribuição binomial negativa quando a observação não está no estado zero-inflacionado.
- **$\alpha$**: Controla a sobredispersão nas contagens positivas. Se $\alpha = 0$, o modelo ZINB se reduz ao modelo ZIP.

### Estimação do Modelo ZINB

A estimação do modelo ZINB envolve a maximização da função de verossimilhança para os dois processos (zero-inflacionado e binomial negativo) simultaneamente. Assim como no modelo ZIP, os parâmetros $\pi$ e $\lambda$ podem ser modelados como funções das covariáveis $X$:

- **$\pi$**: A probabilidade de inflacionamento de zeros pode ser modelada como uma função logística das covariáveis:
  $$
  \pi = \frac{\exp(z' \gamma)}{1 + \exp(z' \gamma)}
  $$
  Onde $z$ são as covariáveis que afetam a probabilidade de inflacionamento e $\gamma$ são os coeficientes a serem estimados.

- **$\lambda$**: A média da distribuição binomial negativa pode ser modelada como uma função exponencial das covariáveis:
  $$
  \lambda = \exp(x' \beta)
  $$
  Onde $x$ são as covariáveis que afetam a contagem e $\beta$ são os coeficientes a serem estimados.

### Aplicações Típicas do Modelo ZINB

O modelo ZINB é particularmente útil em situações onde há um excesso de zeros e a variância das contagens positivas é maior que a média. Exemplos incluem:

- **Número de visitas a um serviço de saúde**: Alguns indivíduos podem nunca visitar um médico, enquanto outros o fazem frequentemente, com variância alta no número de visitas.
- **Incidência de crimes em regiões específicas**: Algumas regiões podem não ter nenhum registro de crime, enquanto outras têm registros frequentes, mas com grande variabilidade.
- **Contagem de espécies em estudos ecológicos**: Algumas áreas podem não conter certas espécies, enquanto outras áreas têm grande variação na contagem dessas espécies.

### Limitações do Modelo ZINB

Apesar de sua flexibilidade, o modelo ZINB possui algumas limitações:

1. **Complexidade Computacional**: A estimação dos parâmetros no modelo ZINB pode ser mais complexa e exigente em termos computacionais do que em modelos mais simples, como o ZIP ou o modelo de Poisson.
2. **Interpretação Complexa**: A interpretação dos coeficientes e a separação dos efeitos entre os dois processos (zero-inflacionado e binomial negativo) pode ser desafiadora.
3. **Necessidade de Dados Ricos**: Para que o modelo ZINB seja efetivo, é necessário um conjunto de dados suficientemente grande e rico em variabilidade, especialmente para identificar corretamente os parâmetros de sobredispersão.

### Alternativas ao Modelo ZINB

Se o modelo ZINB não for adequado, outras alternativas podem ser consideradas:

- **Modelo Hurdle**: Um modelo que também lida com zeros excessivos, mas trata a geração de zeros e contagens positivas como dois processos separados e independentes.
- **Modelo Zero-Inflated Poisson (ZIP)**: Utilizado quando a sobredispersão não é um problema significativo e a contagem positiva pode ser bem modelada por uma distribuição de Poisson.



### Teste de Vuong

O **Teste de Vuong** é um teste estatístico utilizado para comparar a qualidade de ajuste de dois modelos aninhados ou não aninhados, que são usados para modelar os mesmos dados. Esse teste é particularmente útil para comparar modelos de contagem, como o **Modelo Zero-Inflated Poisson (ZIP)** e o **Modelo Zero-Inflated Negative Binomial (ZINB)**, ou para decidir entre um modelo de Poisson e um modelo binomial negativo.

#### Contexto e Aplicação do Teste de Vuong

Quando você tem dois modelos diferentes que podem ser utilizados para modelar os mesmos dados, o Teste de Vuong ajuda a determinar qual modelo se ajusta melhor aos dados. É especialmente útil quando você suspeita que um modelo pode ser superior ao outro, mas ambos os modelos têm boas razões para serem considerados.

O teste compara os logaritmos das funções de verossimilhança dos dois modelos, medindo se há uma diferença significativa entre os dois em termos de ajuste aos dados.

#### Fórmula do Teste de Vuong

O Teste de Vuong é baseado na estatística de Vuong, que é calculada como:

$$
V = \frac{\sqrt{n} \cdot \bar{m}}{s_m}
$$

Onde:
- $n$ é o número de observações.
- $\bar{m}$ é a média das diferenças entre os logaritmos das funções de verossimilhança dos dois modelos para cada observação.
- $s_m$ é o desvio padrão dessas diferenças.

A fórmula para $\bar{m}$ é:

$$
\bar{m} = \frac{1}{n} \sum_{i=1}^{n} ( \ln \mathcal{L}_1(y_i | x_i) - \ln \mathcal{L}_2(y_i | x_i) )
$$

Onde:
- $\mathcal{L}_1$ e $\mathcal{L}_2$ são as funções de verossimilhança dos dois modelos comparados.
- $y_i$ é a observação i.
- $x_i$ são as covariáveis associadas à observação $y_i$.

#### Interpretação do Teste de Vuong

A estatística $V$ segue uma distribuição normal padrão sob a hipótese nula de que ambos os modelos têm o mesmo poder preditivo.

- **Se $V > 1.96$**: O primeiro modelo (modelo 1) se ajusta significativamente melhor aos dados do que o segundo modelo (modelo 2), ao nível de significância de 5%.
- **Se $V < -1.96$**: O segundo modelo (modelo 2) se ajusta significativamente melhor aos dados do que o primeiro modelo (modelo 1), ao nível de significância de 5%.
- **Se $-1.96 \leq V \leq 1.96$**: Não há evidência suficiente para preferir um modelo ao outro; ambos os modelos têm desempenhos semelhantes em termos de ajuste aos dados.

#### Aplicações Práticas

O Teste de Vuong é frequentemente usado nas seguintes situações:

1. **Comparação entre Modelos Zero-Inflated**: Comparar o modelo ZIP com o ZINB para determinar qual modelo lida melhor com os zeros inflacionados e a sobredispersão.
2. **Decisão entre Modelos de Contagem**: Comparar um modelo de Poisson com um modelo binomial negativo para verificar qual modelo descreve melhor os dados de contagem.
3. **Comparação de Modelos Aninhados e Não Aninhados**: Comparar qualquer par de modelos (aninhados ou não aninhados) para determinar qual tem melhor qualidade de ajuste.

#### Limitações do Teste de Vuong

- **Sensibilidade à Especificação do Modelo**: O teste é sensível à especificação dos modelos, e resultados divergentes podem ocorrer se os modelos não forem especificados corretamente.
- **Dependência de Assumptions**: O teste assume que as observações são independentes e identicamente distribuídas (iid). Se essa suposição for violada, os resultados do teste podem ser comprometidos.
- **Comparação Direta**: O teste só pode comparar dois modelos de cada vez, o que pode ser uma limitação em contextos onde vários modelos precisam ser comparados simultaneamente.

#### Exemplo de Uso

Suponha que você tenha ajustado um modelo ZIP e um modelo ZINB aos mesmos dados e deseja determinar qual modelo se ajusta melhor. O Teste de Vuong pode ser utilizado para comparar diretamente os logaritmos das funções de verossimilhança de ambos os modelos, fornecendo evidências sobre qual modelo é superior.

### Aplicação em Python utilizando o rpy2 (R)

In [3]:
import pandas as pd
from rpy2 import robjects
from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import importr

# Ativar a conversão automática entre pandas e R
pandas2ri.activate()

# Carregar o pacote R 'pscl'
pscl = importr('pscl')

# Carregar os dados no Python
df = pd.read_stata("https://github.com/Daniel-Uhr/data/raw/main/cattaneo2.dta")
df['Y'] = df['nprenatal']

# Passar o DataFrame pandas para um DataFrame R
rdf = pandas2ri.py2rpy(df)

# Colocar o DataFrame R no ambiente global do R
robjects.globalenv['rdf'] = rdf

# Definir a fórmula do modelo
formula = 'Y ~ medu + fedu + mage + deadkids + alcohol'

# Ajustar o modelo ZINB usando o pacote pscl
robjects.r(f'''
zinb_model <- zeroinfl({formula}, data=rdf, dist="negbin")
''')
# Obter o resumo do modelo ZINB
print(robjects.r('summary(zinb_model)'))

# Ajustar o modelo ZIP usando o pacote pscl
robjects.r(f'''
zip_model <- zeroinfl({formula}, data=rdf, dist="poisson")
''')
# Obter o resumo do modelo ZIP
print(robjects.r('summary(zip_model)'))

# Realizar o Teste de Vuong diretamente em R
vuong_test = robjects.r('vuong(zinb_model, zip_model)')

# Obter o resumo do Teste de Vuong
print(vuong_test)



Call:
zeroinfl(formula = Y ~ medu + fedu + mage + deadkids + alcohol, data = rdf, 
    dist = "negbin")

Pearson residuals:
     Min       1Q   Median       3Q      Max 
-3.43479 -0.53576  0.06026  0.54287  7.11062 

Count model coefficients (negbin with log link):
              Estimate Std. Error z value Pr(>|z|)    
(Intercept)  1.9923310  0.0274035  72.704  < 2e-16 ***
medu         0.0146850  0.0023345   6.290 3.17e-10 ***
fedu         0.0089467  0.0015902   5.626 1.84e-08 ***
mage         0.0040253  0.0009114   4.417 1.00e-05 ***
deadkids    -0.0178692  0.0104653  -1.707  0.08773 .  
alcohol     -0.0464488  0.0270871  -1.715  0.08638 .  
Log(theta)   6.9966781  2.1370895   3.274  0.00106 ** 

Zero-inflation model coefficients (binomial with logit link):
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)  0.03559    0.59479   0.060    0.952    
medu        -0.05506    0.04107  -1.341    0.180    
fedu        -0.11754    0.02384  -4.930 8.23e-07 ***
mage        -0.094

Dado o p-valor de 0.44439, que é bem acima do nível de significância comum (0.05), não há evidência estatística para preferir o modelo ZINB ao modelo ZIP. Em outras palavras, ambos os modelos parecem ajustar os dados de maneira semelhante, e não há uma vantagem clara em usar um sobre o outro com base no Teste de Vuong.

### Modelo Tobit

O **modelo Tobit** é utilizado em situações onde a variável dependente de uma regressão linear é censurada, ou seja, só é observada dentro de um intervalo específico. Esse modelo é particularmente útil quando há um limite superior ou inferior além do qual as observações não são registradas ou são reportadas como um valor fixo, como zero.

#### Cenário de Aplicação

Em um conjunto de dados $(y_i, x_i)$, $i=1, \ldots, N$, as variáveis explicativas $x_i$ são completamente observadas e exógenas, mas a variável dependente $y_i$ pode não ser completamente observada. Por exemplo, imagine um estudo sobre os gastos domiciliares com determinado bem. Se alguns domicílios não compram o bem, o gasto observado será zero. Neste caso, $y_i$ pode ser tratado como censurado.

Nesse contexto, podemos interpretar que existe uma variável latente $y^*$ que representa a verdadeira demanda por esse bem. No entanto, essa demanda só se manifesta quando ultrapassa um certo limite, $L$. Se a demanda for menor ou igual a esse limite, observamos um valor censurado.

#### Estrutura do Modelo Tobit

A regressão Tobit é especificada em termos de uma variável latente $y_i^*$, que segue a seguinte forma:

$$ y_i^* = x_i' \beta + \epsilon_i, \quad \epsilon_i \sim N(0, \sigma^2) $$

Onde:
- $y_i^*$ é a variável latente (não observada diretamente).
- $x_i$ é o vetor de variáveis explicativas.
- $\beta$ é o vetor de coeficientes a serem estimados.
- $\epsilon_i$ são os erros, assumidos como normalmente distribuídos com média zero e variância constante $\sigma^2$.

A variável observada $y_i$ está relacionada à variável latente $y_i^*$ da seguinte forma:

- **Censura à Esquerda**: 
  - $y_i = y_i^*$ se $y_i^* > L$
  - $y_i = L$ se $y_i^* \leq L$
  
  Aqui, $L$ é o ponto de censura inferior.

- **Censura à Direita**: 
  - $y_i = y_i^*$ se $y_i^* < U$
  - $y_i = U$ se $y_i^* \geq U$
  
  Aqui, $U$ é o ponto de censura superior.

#### Função de Verossimilhança

A função de verossimilhança para o modelo Tobit combina as probabilidades associadas às observações censuradas e não censuradas. Para uma censura à esquerda:

- Para as observações não censuradas ($y_i > L$), a contribuição para a verossimilhança é a densidade normal:
  
  $$ f(y_i | x_i) = \frac{1}{\sigma} \phi\left(\frac{y_i - x_i' \beta}{\sigma}\right) $$

  Onde $\phi(.)$ é a função densidade da distribuição normal padrão.

- Para as observações censuradas ($y_i = L$), a contribuição para a verossimilhança é a probabilidade acumulada:

  $$ F(L | x_i) = \Phi\left(\frac{L - x_i' \beta}{\sigma}\right) $$

  Onde $\Phi(.)$ é a função de distribuição acumulada da normal padrão.

A função de verossimilhança total é a combinação dos termos para todas as observações.

#### Considerações sobre o Modelo Tobit

O modelo Tobit pressupõe normalidade dos erros e homocedasticidade. No entanto, em algumas aplicações, como em dados de consumo, pode ser mais adequado modelar os dados como lognormais, especialmente se os valores observados forem sempre positivos:

$$ y_i^* = \exp(x_i' \beta + \epsilon_i), \quad \epsilon_i \sim N(0, \sigma^2) $$

Nesse caso, a variável observada $y_i$ é dada por:

- $y_i = y_i^*$ se $\ln y_i^* > \gamma$
- $y_i = 0$ se $\ln y_i^* \leq \gamma$

Aqui, $\gamma$ é o ponto de censura em termos do logaritmo da variável latente.

#### Diagnóstico e Validade do Modelo Tobit

A consistência do estimador Tobit depende das seguintes condições:

1. **Normalidade dos Erros**: A suposição de normalidade pode ser verificada usando testes como o `sktest`. Se os erros não forem normalmente distribuídos, as estimativas podem ser inconsistentes.

2. **Homocedasticidade**: A suposição de variância constante (homocedasticidade) é essencial para a consistência do modelo. Isso pode ser testado usando o teste de Lagrange Multiplier (LM). Se a homocedasticidade não for verificada, o modelo pode produzir estimativas enviesadas.

Mesmo que o modelo Tobit produza estimativas que parecem razoáveis, é crucial realizar diagnósticos rigorosos para garantir que as suposições subjacentes sejam válidas.

#### Alternativas ao Modelo Tobit

Caso as suposições do modelo Tobit sejam violadas, é importante considerar alternativas:

- **Modelo Truncado**: Utilizado quando as observações abaixo (ou acima) de um certo valor são completamente excluídas do conjunto de dados, ao invés de serem censuradas.

- **Modelo de Heckman**: Usado para lidar com problemas de seleção amostral. Esse modelo é útil quando a decisão de observar ou não a variável dependente está correlacionada com fatores que também afetam o resultado. Ele corrige o viés de seleção e fornece estimativas consistentes.



### Tobit

O modelo Tobit é relevante quando a variável dependente de uma regressão linear é observada somente para algum intervalo. Os dados $(y_i, x_i)$, $i=1, \ldots, N$ assumem que $x_i$ é completamente observado (exógeno), mas $y_i$ nem sempre é observado. Ou seja, alguns $y_i$ são zero.

Uma interpretação é que os valores zero são observações censuradas (no sentido de que há perda de informação, podem ser “missings”). Por exemplo, considere uma variável latente (não observável) domiciliar, $y^*$, por demanda de bens. Ela não é expressa até que seja feita a compra em determinado nível, denotado por $L$, seja ultrapassado. Assim, $L$ pode adotar um valor maior que zero, mesmo que desconhecido.

A distribuição tem suporte sobre o intervalo de $-\infty$ a $+\infty$, mas somente observamos valores no intervalo $[L, U]$, ou seja, $L$ é o ponto de corte inferior, e $U$ o ponto de corte superior.

#### Estrutura do Modelo Tobit

A regressão de interesse é especificada como uma variável latente, $y^*$:

$$ y_i^* = x_i' \beta + \epsilon_i, \quad i = 1, \ldots, N $$

Onde $\epsilon_i \sim N(0, \sigma^2)$ e $x_i$ denota o vetor $(K \times 1)$ exógeno e completamente observado. Se $y_i^*$ fosse observado, estimaríamos por OLS.

A variável observável $y_i$ é relacionada à variável latente $y_i^*$ através da regra de observação:

$$ y_i = \begin{cases} 
y_i^* & \text{se } y_i^* > L \\
L & \text{se } y_i^* \leq L 
\end{cases} $$

A probabilidade de uma observação ser censurada é $ \Pr(y_i^* \leq L) = \Pr(x_i' \beta + \epsilon_i \leq L) = \Phi \left(\frac{L - x_i' \beta}{\sigma} \right)$, onde $\Phi(.)$ é a Função de Distribuição Acumulada Normal Padrão.

A censura também pode ocorrer pela direita:

$$ y_i = \begin{cases} 
y_i^* & \text{se } y_i^* < U \\
U & \text{se } y_i^* \geq U 
\end{cases} $$

#### Considerações sobre o Modelo Tobit

Modelos Tobit assumem normalidade. Entretanto, dados de consumo são melhores modelados como lognormal. Complicações adicionais: threshold não zero e $y$ lognormal. Então:

$$ y_i^* = \exp(x_i' \beta + \epsilon_i), \quad \epsilon_i \sim N(0, \sigma^2) $$

Onde observamos que:

$$ y_i = \begin{cases} 
y_i^* & \text{se } \ln y_i^* > \gamma \\
0 & \text{se } \ln y_i^* \leq \gamma 
\end{cases} $$

Aqui se conhece que $y_i = 0$ quando os dados são censurados, e em geral $\gamma \ne 0$. Podemos rodar o Tobit com a opção `ll(#)`, onde a variável dependente é $\ln y$, e o valor do threshold # é igual ao valor mínimo não censurado de $\ln y$.

Ocorre que muitos valores da variável dependente originalmente são zeros, e ao aplicarmos o $\ln(.)$ teremos dados missing. Além disso, o menor valor pode ser 1, onde $\ln(1)$ é zero. E `ll(0)` tratará a observação como censurada.

#### Validade do Tobit

A consistência do estimador Tobit depende de:
- Especificação Correta do Modelo
- Exogeneidade dos Regressores
- **Normalidade**
- **Homocedasticidade**
  
Apesar das estimações aparentemente satisfatórias, os diagnósticos revelam a fraqueza do modelo.


Exemplo Aplicado Tobit: Limite de censura constante

Os administradores universitários querem saber a relação entre a média das notas do ensino médio (GPA) e o desempenho dos alunos na faculdade. O arquivo gpa.dta contém dados fictícios sobre uma coorte de 4.000 estudantes universitários. O GPA da faculdade (gpa2) e o GPA do ensino médio (hsgpa) são medidos em uma escala contínua entre zero e quatro. O resultado de interesse é o GPA universitário do estudante. Porém, por razões de confidencialidade, os GPAs abaixo de 2.0 são reportados como 2.0. Em outras palavras, o resultado é censurado à esquerda. Acreditamos que o GPA também é uma função do logaritmo da renda dos pais do estudante (pincome) e se o estudante participou ou não de um programa de habilidades de estudo durante a faculdade (program).

In [43]:
import requests
import sys
# URL do arquivo tobit.py
url = 'https://raw.githubusercontent.com/Daniel-Uhr/estimators/main/tobit.py'

# Fazer o download do arquivo
response = requests.get(url)
code = response.text
# Executar o código do arquivo baixado
exec(code)

In [44]:
import pandas as pd
import numpy as np

# Carregar os dados
df = pd.read_stata("https://github.com/Daniel-Uhr/data/raw/main/gpa.dta")

# Preparar as variáveis independentes e a variável dependente censurada
x = df[['hsgpa', 'pincome', 'program']]

# Converter a variável categórica 'program' em variáveis dummy
x = pd.get_dummies(x, drop_first=True)

y = df['gpa2']

# sumarizar a variável y
print(y.describe())

count    4000.000000
mean        2.571925
std         0.585179
min         2.000000
25%         2.000000
50%         2.426000
75%         2.981000
max         4.000000
Name: gpa2, dtype: float64


In [45]:
# Considerando censura superior em 4.0 e inferior em 2.0
cens = pd.Series(0, index=df.index)
cens[y >= 4.0] = 1  # Censura à direita
cens[y <= 2.0] = -1  # Censura à esquerda


# Criar e ajustar o modelo Tobit
model = TobitModel(fit_intercept=True)
model.fit(x, y, cens)

# As previsões e a avaliação do modelo são opcionais
y_pred = model.predict(x)
mae = model.score(x, y, scoring_function=mean_absolute_error)
print("Mean Absolute Error:", mae)

  Tobit Regression Results
Dep. Variable:                     gpa2
Model:                            Tobit
Method:                           Maximum Likelihood
Date:                             dom, 11 ago 2024
Time:                             18:57:33
No. Observations:                 4000
No. Censored Observations:        1310
No. Uncensored Observations:      2690
                 coef   std err          z  P>|z|    [0.025    0.975]
const        0.681005  0.049097  13.870657    0.0  0.584775  0.777234
hsgpa        0.681005  0.012887  52.844776    0.0  0.655746  0.706263
pincome      0.329633  0.007449  44.254492    0.0  0.315034  0.344232
program_Yes  0.571642  0.015109  37.835526    0.0  0.542029  0.601255
Sigma (scale):                    0.4151
Log-likelihood:                   2134.6044
Number of Iterations:             15
Mean Absolute Error: 0.8694328578311515


Com base nas estatísticas descritivas, é razoável considerar a censura superior em 4.0, mas a censura inferior em 2.0 dependerá do contexto e de um entendimento adicional dos dados. Se há um conhecimento prévio ou uma razão lógica para acreditar que 2.0 é um limite inferior, então você deve configurar o modelo para considerar isso. Caso contrário, a censura inferior pode não ser necessária.


A regressão Tobit relata os coeficientes para o modelo de regressão latente. Assim, podemos interpretar os coeficientes da mesma forma que interpretaríamos os coeficientes de uma regressão OLS. Por exemplo, a participação em um programa de habilidades de estudo aumenta a expectativa do GPA não censurado em 0,56 pontos.

In [47]:
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.stats.diagnostic import het_breuschpagan

# Supondo que 'model' é o seu objeto TobitModel ajustado
# e que 'y_pred' são as previsões do modelo.

# 1. Resíduos do modelo
residuals = y - y_pred

# 2. Teste de Shapiro-Wilk para Normalidade
shapiro_test = stats.shapiro(residuals)
print("Shapiro-Wilk Test:")
print(f"Statistic: {shapiro_test.statistic}, p-value: {shapiro_test.pvalue}")

# 3. Teste de Breusch-Pagan para Homocedasticidade
# Adicionando uma constante aos valores ajustados
exog_het = sm.add_constant(y_pred)
bp_test = het_breuschpagan(residuals, exog_het)
print("Breusch-Pagan Test:")
print(f"Lagrange multiplier statistic: {bp_test[0]}, p-value: {bp_test[1]}")
print(f"F-statistic: {bp_test[2]}, F-test p-value: {bp_test[3]}")

Shapiro-Wilk Test:
Statistic: 0.9982970356941223, p-value: 0.00026581567362882197
Breusch-Pagan Test:
Lagrange multiplier statistic: 1021.5150332114224, p-value: 3.7820086335399504e-224
F-statistic: 1371.1726425742825, F-test p-value: 2.488799594838788e-258


os testes de Shapiro-Wilk e Breusch-Pagan indicam que os pressupostos de normalidade dos resíduos e homocedasticidade dos resíduos foram violados

### Modelo de Seleção de Heckman

O **Modelo de Seleção de Heckman** é uma abordagem para corrigir o viés de seleção amostral que ocorre quando a amostra observada não é aleatória, mas sim selecionada de acordo com alguma regra ou decisão, o que pode gerar resultados enviesados se não for tratado adequadamente. O modelo é amplamente utilizado em situações onde a decisão de participar em um evento ou a decisão de consumo pode estar correlacionada com o volume do resultado observado.

#### Cenário do Modelo

Considerando o exemplo anterior do modelo Tobit, temos duas decisões que precisam ser modeladas:

1. **Decisão de gastar (participação)**: Se o indivíduo decide ou não participar do mercado (por exemplo, decidir se irá gastar ou não).
2. **Volume de gastos (resultado)**: Quanto o indivíduo gasta, dado que ele decidiu participar.

Se tratarmos essas duas decisões como independentes, podemos cometer um erro, já que a decisão de participar pode influenciar o volume de gastos, criando um viés de seleção amostral.

### Viés de Seleção Amostral

Mesmo depois de controlar por variáveis explicativas, o nível de consumo positivo observado pode não ser selecionado aleatoriamente da população. Em outras palavras, os indivíduos "se selecionam" para participar do mercado de consumo, o que leva ao **viés de seleção amostral**.

### Procedimento de Heckman

Para corrigir o viés de seleção amostral, utilizamos o **procedimento de Heckman**, que é um modelo de seleção em duas partes:

1. **Parte 1: Modelo de Seleção (Probit)**:
   - Primeiramente, estimamos a probabilidade de participação utilizando um modelo Probit.
   - O modelo Probit estima a probabilidade de um resultado binário (participação ou não).

2. **Parte 2: Modelo de Resultado (OLS com Correção)**:
   - Em seguida, utilizamos os resultados da primeira parte para ajustar o modelo de resultado.
   - O modelo OLS é corrigido pela inclusão de uma estimativa do regressor omitido, conhecido como **razão inversa de Mills**.

### Derivação do Modelo

Para o caso de truncamento à esquerda, em zero, somente observamos $y$ se $y^* > 0$. Então:

$$ E[y] = E[y^* | y^* > 0] $$

Podemos decompor isso da seguinte maneira:

$$ E[y]= E[x' \beta + \epsilon | x' \beta + \epsilon > 0] $$

$$ E[y]= E[x' \beta | x' \beta + \epsilon > 0] + E[\epsilon | x' \beta + \epsilon > 0] $$

$$ E[y]= x' \beta + E[\epsilon | \epsilon > -x' \beta] $$

Para uma variável padronizada, $z \sim N(0,1)$, truncada à esquerda em zero, tem-se que seu primeiro momento é:

$$ E[z | z > -c] = \frac{\phi(c)}{\Phi(c)} $$

Então, temos que:

$$ E[y] = E[y^* | y^* > 0] = x' \beta + E[\epsilon | \epsilon > -x' \beta] $$

Para o último termo:

$$ E[\epsilon | \epsilon > -x' \beta] = \sigma E\left[\frac{\epsilon}{\sigma} \Bigg| \frac{\epsilon}{\sigma} > \frac{-x' \beta}{\sigma} \right] $$

$$ = \sigma \frac{\phi\left(\frac{x' \beta}{\sigma}\right)}{\Phi\left(\frac{x' \beta}{\sigma}\right)} $$

$$ = \sigma \lambda\left(\frac{x' \beta}{\sigma}\right) $$

Onde $\lambda(\cdot)$ é chamada a razão inversa de Mills.












### Modelo Heckman de Dois Passos

No **procedimento de dois passos de Heckman** (Heckit), corrigimos a regressão OLS adicionando uma estimativa do regressor omitido $ \lambda(x_1' \beta_1) $. Usando os valores positivos de $ y_2 $, estimamos pelo modelo OLS:

$$ y_{2i} = x_{2i}' \beta_2 + \sigma_{12} \lambda(x_1' \hat{\beta_1}) + v_i $$

Aqui, $ \hat{\beta_1} $ são os coeficientes obtidos no primeiro passo (regressão Probit). Além disso, $ x_1 $ e $ x_2 $ podem ser diferentes, permitindo a inclusão de variáveis que identificam a participação, conhecidas como **variáveis de exclusão**.

### Notas Importantes:

- **Restrição de Exclusão**: A inclusão de uma variável que afeta a participação (decisão de gastar), mas não afeta diretamente o volume de gastos, é crucial para a identificação do modelo. Isso é conhecido como uma **restrição de exclusão**.
- **Renda como Variável de Exclusão**: No exemplo, a renda é utilizada como uma variável de exclusão, mas é importante observar que a renda também pode ser uma variável de resultado, o que pode complicar sua utilização como variável de exclusão.

### Considerações Finais

O **Modelo de Seleção de Heckman** é uma ferramenta poderosa para lidar com o viés de seleção amostral em econometria. Ele permite modelar de forma mais realista situações onde a decisão de participar em um evento e o resultado desse evento estão inter-relacionados. A correção de Heckman ajuda a evitar a distorção nos coeficientes e a obter estimativas consistentes.

### Exercícios e Discussão

- **Exercício Prático**: Use um software econométrico, como o Stata, para aplicar o procedimento de Heckman a um conjunto de dados com viés de seleção.
- **Discussão**: Discuta a importância da escolha de variáveis de exclusão adequadas e os desafios na identificação do modelo de seleção de Heckman.

Essa aula fornece uma base sólida tanto para a compreensão teórica quanto para a aplicação prática do modelo de seleção de Heckman. Se precisar de mais exemplos ou aprofundamento em algum aspecto específico, estou à disposição!

In [33]:
import sys
import os
# Adicione o caminho do script heckman.py ao Python Path
sys.path.append(os.path.join(os.getcwd(), 'estimators'))

import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from heckman import Heckman

In [34]:
df3 = pd.read_stata("https://github.com/Daniel-Uhr/data/raw/main/womenwk.dta")

Roda somente o two-step Heckman.

In [35]:
# Definir as variáveis endógenas, exógenas e de seleção
endog = df3['wage']
exog = sm.add_constant(df3[['education', 'age']])
exog_select = sm.add_constant(df3[['married', 'children', 'education', 'age']])

# Ajustar o modelo Heckman
model = Heckman(endog, exog, exog_select)
results = model.fit(method='twostep')

# Exibir os resultados
print(results.summary())

       Heckman Regression Results      
Dep. Variable:                     wage
Model:                          Heckman
Method:                Heckman Two-Step
Date:                  dom, 11 ago 2024
Time:                          18:47:11
No. Total Obs.:                    2000
No. Censored Obs.:                  657
No. Uncensored Obs.:               1343
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.7340      1.248      0.588      0.557      -1.713       3.181
education      0.9825      0.054     18.235      0.000       0.877       1.088
age            0.2119      0.022      9.608      0.000       0.169       0.255
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.4674      0.193    -12.813      0.000      -2.845      -2.090
married 

A tabela 1 - Estimativas da Equação de Resposta (Resultado)
* Constante (const): O intercepto da equação de resposta não é significativo, indicando que, quando todas as variáveis explicativas são zero, o salário (wage) esperado não é significativamente diferente de zero.
* Education (education): A educação tem um coeficiente positivo e altamente significativo, sugerindo que a educação está positivamente associada ao salário.
* Age (age): A idade também tem um coeficiente positivo e significativo, indicando que a idade está positivamente correlacionada com o salário.

A tabela 2 - Estimativas da Equação de Seleção (Participação)
* Constante (const): O intercepto da equação de seleção é significativo e negativo, o que pode indicar a linha de base para a decisão de participar no mercado de trabalho.
* Married (married): O fato de ser casado aumenta significativamente a probabilidade de participar no mercado de trabalho.
* Children (children): Ter filhos também aumenta significativamente a probabilidade de participação no mercado de trabalho.
* Education (education) e Age (age): Tanto a educação quanto a idade aumentam a probabilidade de participação no mercado de trabalho, com coeficientes significativos.

A tabela 3 - Estimativas do Modelo de Heckman
* IMR (Lambda): O coeficiente da razão inversa de Mills é significativo, indicando que há viés de seleção amostral presente. A correção de Heckman é, portanto, justificada.
* Rho (rho): Representa a correlação entre os erros da equação de seleção e da equação de resultado. Um valor de 0,673 sugere uma correlação substancial entre as duas equações.
* Sigma (sigma): Estima o desvio padrão dos erros na equação de resultado. Um sigma maior indica maior variabilidade nos salários observados, dado o viés de seleção.


Considerações Finais
Viés de Seleção Amostral:

A significância de Lambda (IMR) sugere que o modelo de Heckman corrigiu adequadamente o viés de seleção presente na amostra.
Interpretação Prática:

O modelo mostra que tanto a decisão de trabalhar quanto o salário são influenciados por variáveis como educação, idade, estado civil e presença de filhos.
As duas equações (seleção e resultado) não são independentes, o que é capturado pelo valor de rho.
Ajuste de Modelos e Aplicações:

O modelo de Heckman é ideal em situações onde a amostra observada não é representativa da população total devido à seleção amostral, como no caso das decisões de participação no mercado de trabalho.
Esse exemplo mostra como o modelo de Heckman pode ser aplicado para lidar com o viés de seleção amostral em dados reais. Se precisar de mais detalhes ou exemplos adicionais, estou à disposição!